In [2]:
import boto3
s3 = boto3.resource('s3')

In [5]:
s3.Bucket('recommendationsystemshubham').download_file('Nearest Neighbors Rec Engine/BX-Users.csv','datasets/BX-Users.csv')

In [6]:
s3.Bucket('recommendationsystemshubham').download_file('Nearest Neighbors Rec Engine/BX-Book-Ratings.csv','datasets/BX-Book-Ratings.csv') 

In [7]:
s3.Bucket('recommendationsystemshubham').download_file('Nearest Neighbors Rec Engine/BX-Books.csv','datasets/BX-Books.csv')

In [13]:
import pandas as pd
dataFile='datasets/BX-Book-Ratings.csv'
data=pd.read_csv(dataFile,sep=";",header=0,encoding = "ISO-8859-1", error_bad_lines=False,names=["user","isbn","rating"])

In [14]:
data.head()

user        isbn  rating
0  276725  034545104X       0
1  276726  0155061224       5
2  276727  0446520802       0
3  276729  052165615X       3
4  276729  0521795028       6

In [18]:
bookFile='datasets/BX-Books.csv'
books=pd.read_csv(bookFile,sep=";",header=0,error_bad_lines=False, encoding = "ISO-8859-1",usecols=[0,1,2],index_col=0,names=['isbn',"title","author"])

In [19]:
books.head()

title  \
isbn                                                            
0195153448                                Classical Mythology   
0002005018                                       Clara Callan   
0060973129                               Decision in Normandy   
0374157065  Flu: The Story of the Great Influenza Pandemic...   
0393045218                             The Mummies of Urumchi   

                          author  
isbn                              
0195153448    Mark P. O. Morford  
0002005018  Richard Bruce Wright  
0060973129          Carlo D'Este  
0374157065      Gina Bari Kolata  
0393045218       E. J. W. Barber

In [20]:
def bookMeta(isbn):
    title = books.at[isbn,"title"]
    author = books.at[isbn,"author"]
    return title, author
bookMeta("0671027360")

('Angels &amp; Demons', 'Dan Brown')

In [21]:
data = data[data["isbn"].isin(books.index)]

In [22]:
def faveBooks(user,N):
    userRatings = data[data["user"]==user]
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N] 
    sortedRatings["title"] = sortedRatings["isbn"].apply(bookMeta)
    return sortedRatings

In [23]:
faveBooks(204622,5)

user        isbn  rating  \
844955  204622  0967560500      10   
844935  204622  0671027360      10   
844926  204622  0385504209      10   
844958  204622  097173660X       9   
844920  204622  0060935464       9   

                                                   title  
844955        (Natural Hormonal Enhancement, Rob Faigin)  
844935                  (Angels &amp; Demons, Dan Brown)  
844926                    (The Da Vinci Code, Dan Brown)  
844958  (Life After School Explained, Cap &amp; Compass)  
844920               (To Kill a Mockingbird, Harper Lee)

In [24]:
data.shape

(1031175, 3)

In [25]:
usersPerISBN = data.isbn.value_counts()
usersPerISBN.head(10)

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
044023722X     647
0142001740     615
067976402X     614
0671027360     586
0446672211     585
Name: isbn, dtype: int64

In [26]:
usersPerISBN.shape

(270170,)

In [27]:
ISBNsPerUser = data.user.value_counts()

In [28]:
ISBNsPerUser.shape

(92107,)

In [29]:
data = data[data["isbn"].isin(usersPerISBN[usersPerISBN>10].index)]

In [30]:
data = data[data["user"].isin(ISBNsPerUser[ISBNsPerUser>10].index)]